In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySparkShell") \
    .getOrCreate()

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as sf

In [4]:
import pandas as pd

In [5]:
import plotly.graph_objects as go

In [6]:
from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
import seaborn as sns

In [8]:
Countries = spark.read.csv('covid19',inferSchema=True, header=True)
Countries=Countries.orderBy("date", ascending=False)

In [9]:
Countries.createOrReplaceTempView("Countries")
Countries=spark\
.sql(\
     "SELECT location, total_cases_per_million \
     FROM Countries \
     Where date='2020-04-27'")
Countries=Countries.orderBy("total_cases_per_million", ascending=False)

In [10]:
Countries=Countries.withColumn("RoundTotalCases", lit(Countries.total_cases_per_million/1000))

In [11]:
Countries = Countries.selectExpr("cast(location as string) location","cast(total_cases_per_million as int) total_cases_per_million","cast(RoundTotalCases as int) RoundTotalCases")
Countries=Countries.groupBy("RoundTotalCases") \
.count() \
.orderBy("RoundTotalCases", ascending=False)


In [12]:
Countries=Countries.na.drop()

In [13]:
Countries=Countries.withColumn("RoundTotalCasesPlus", lit(Countries.RoundTotalCases+1))

In [14]:
Countries = Countries.selectExpr("cast(count as int) count","cast(RoundTotalCases as string) RoundTotalCases","cast(RoundTotalCasesPlus as string) RoundTotalCasesPlus")

In [15]:
Countries = Countries\
.withColumn('Range',sf.concat(sf.lit('('),sf.col('RoundTotalCases'),sf.lit('-'), sf.col('RoundTotalCasesPlus'),sf.lit('k)')))
Countries = Countries.selectExpr("cast(count as int) cnt","cast(Range as String) Range")

In [16]:
temp=Countries.withColumn("tempo", lit(1))

In [17]:
temp=temp.groupBy("tempo").sum()

In [18]:
temp=temp.withColumnRenamed("sum(cnt)","NumOfCountries")

In [19]:
Countries.createOrReplaceTempView("T1")
temp.createOrReplaceTempView("T2")

In [20]:
Countries=spark\
.sql(\
     "SELECT T2.NumOfCountries, T1.Range, T1.cnt \
     FROM T1, T2")


In [21]:
finel = Countries.selectExpr("cast(NumOfCountries as int) NumOfCountries","cast(Range as String) Range","cast(cnt as int) cnt")


In [22]:
finel=finel.withColumn("Rate", lit((finel.cnt/finel.NumOfCountries)*100))

In [23]:
finel.show()
finel

+--------------+--------+---+------------------+
|NumOfCountries|   Range|cnt|              Rate|
+--------------+--------+---+------------------+
|           205|  (0-1k)|160| 78.04878048780488|
|           205| (9-10k)|  1|0.4878048780487805|
|           205|  (1-2k)| 22|10.731707317073171|
|           205|  (3-4k)|  8| 3.902439024390244|
|           205|  (4-5k)|  2| 0.975609756097561|
|           205|  (2-3k)|  8| 3.902439024390244|
|           205|  (5-6k)|  2| 0.975609756097561|
|           205|(15-16k)|  1|0.4878048780487805|
|           205|(11-12k)|  1|0.4878048780487805|
+--------------+--------+---+------------------+



DataFrame[NumOfCountries: int, Range: string, cnt: int, Rate: double]

In [24]:
densityData=finel.selectExpr("cast(Range as String) Range","cast(Rate as double) Rate")
densityData.show()

+--------+------------------+
|   Range|              Rate|
+--------+------------------+
|  (0-1k)| 78.04878048780488|
| (9-10k)|0.4878048780487805|
|  (1-2k)|10.731707317073171|
|  (3-4k)| 3.902439024390244|
|  (4-5k)| 0.975609756097561|
|  (2-3k)| 3.902439024390244|
|  (5-6k)| 0.975609756097561|
|(15-16k)|0.4878048780487805|
|(11-12k)|0.4878048780487805|
+--------+------------------+

